In [157]:
%load_ext autoreload
%autoreload 2


%matplotlib inline
%cd C:\MAD4AG

from tqdm.auto import tqdm
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\MAD4AG


In [158]:
df_survey_act = pd.read_pickle(f'./dbs/survey_act_weekday.pkl')
df_survey_twins = pd.read_pickle(f'./dbs/intermediate/df_survey_twins.pkl')


In [159]:
df_survey_act = df_survey_act[df_survey_act['sub_id'].isin(df_survey_twins['sub_id'])]


In [160]:
df_survey_act = pd.merge(df_survey_act, df_survey_twins[['sub_id','act_seq_simp', 'modified']], on='sub_id', how='left')

In [161]:
df_survey_act_modified = df_survey_act[df_survey_act.modified==1]

df_survey_act_modified['del']=0

In [162]:
def seq_modifier(data):
    data = data.iloc[:,:]
    counter = 0
    rows= [0]
    for i in range(len(data) - 1, 0, -1):
        #rows += [i]
        #rows.remove([i])
        if data.iloc[i,8] in ['Home', 'Work', 'School']:
            counter = 0
            if data.iloc[i,8] == data.iloc[i-1,8]:
                #data.drop(index=data.iloc[i, :].index.tolist(), inplace=True)
                #data= data.iloc[i-1,8]
                data.iloc[i,17] =1
                data.iloc[i-1,7] = data.iloc[i,7]
                # print(rows)
                #rows = rows.del(i)
                #rows -= [i]
                #rows.remove(i)
        elif data.iloc[i,8] == 'Other':
            counter += 1
            if counter > 3:
                data.iloc[i,17] =1
                data.iloc[i+counter-3,6] = data.iloc[i,6]

    #         else:
    #             rows= rows.append(i)
    #data = data.iloc[rows,:]

    return data


In [164]:
tqdm.pandas()
df_survey_act_modified = df_survey_act_modified.groupby('sub_id').progress_apply(seq_modifier)

  0%|          | 0/2254 [00:00<?, ?it/s]

In [167]:
df_survey_act_modified = df_survey_act_modified[df_survey_act_modified['del']==0]

df_survey_act_modified.drop(columns=['del'],inplace=True)

In [169]:
df_survey = pd.concat([df_survey_act[df_survey_act.modified==0], df_survey_act_modified], ignore_index=True)


In [170]:
df_survey.to_pickle(f'./dbs/intermediate/survey_act_weekday_modified.pkl')